In [7]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib #send email
import pandas as pd
import json

In [101]:
url = 'https://eu-site-api.byd.com/byd-api/eu/dealer/getFindDealer?country=DE&type=sales'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page = requests.get(url, headers = headers)

soup1 = BeautifulSoup(page.text, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

In [103]:
json_text = soup2.get_text()

data = json.loads(json_text)
df = pd.json_normalize(data)

In [105]:
pd.set_option('display.max_columns', None)
df

,country,type,organizationCode,organizationName,status,statusTrans,website,photoUrl,parentCode,lng,lat,zipCode,phoneNumber,mail,feature,address,createdTime,updatedTime,business,dtaSigned,businessHours.hours,businessHours.close,businessHours.closedTrans,businessHours.flag,businessHours.weektime
0,DE,4,DES0029,BYD Lübeck Store,operation,open,None,None,DED005,10.680970,53.849381,23560,+49 (0)451 8801 0,info@auto-senger.de,salesAndAfterSales,"Berliner Str. 4-6, 23560, Lübeck, Schleswig-Ho...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '8:00 am-6:00 pm'..."
1,DE,4,DES0026,BYD Oldenburg Store,operation,open,https://www.senger-mobility.de/marken/byd-buil...,None,DED005,8.178251,53.150720,26129,+49 44177070,info.oldenburg-pkw@auto-senger.de,salesAndAfterSales,"Ammerländer Heerstraße 166-176, 26129, Oldenbu...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '8:00 am-6:00 pm'..."
2,DE,4,DES0014,BYD Magdeburg Store,operation,open,https://sternauto-gruppe.de/standort/magdeburg...,https://eucrm.byd.com/oss/crm/fd338f37-ea4a-46...,DED002,11.572244,52.172475,39130,+49 391 72950,magdeburg@sternauto.de,salesAndAfterSales,"Am großen Silberberg 3, 39130, Magdeburg, Saxo...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '8:00 am-7:00 pm'..."
3,DE,4,DES0020,BYD Bielefeld Store,operation,open,https://www.senger-mobility.de/wir-sind-senger...,https://eucrm.byd.com/oss/crm/f8ccc4ca-ba4c-4b...,DED005,8.560993,52.033919,33609,+49 521 9273950,BYD@Auto-Senger.de,salesAndAfterSales,"Eckendorfer Straße 40, 33609, Bielefeld, North...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '8:00 am-6:00 pm'..."
4,DE,4,DES0015,BYD Erfurt Store,operation,open,https://sternauto-gruppe.de/standort/erfurt,https://eucrm.byd.com/oss/crm/f5dc4c7c-a5e0-43...,DED002,10.970465,50.972789,99092,+49 361 21500,info@russ.janot.de,salesAndAfterSales,"Binderslebener Landstraße 92, 99092, Erfurt, T...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '8:00 am-6:00 pm'..."
5,DE,4,DES0019,BYD Stuttgart Pioneer Store,operation,open,byd.com/de,https://eucrm.byd.com/oss/crm/1576d71b-8a9a-41...,DED010,9.173147,48.775983,70174,+49 4074306251,info.Stuttgart@byd.com,sales,"Theodor-Heuss-Straße 23, 70174, Stuttgart, Bad...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '10:00 am-6:00 pm..."
6,DE,4,DES0023,BYD Saarbrücken Store,operation,open,https://hedinautomotive.de/de/pages/standorte_byd,None,DED006,7.025680,49.225445,66121,+49 681 968080,dialog@byd.store,salesAndAfterSales,"Brebacher Landstraße 1, 66121, Saarbrücken, Sa...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '9:00 am-5:30 pm'..."
7,DE,4,DES0010,BYD Ulm Store,operation,open,https://www.reisacher.de/byd/,https://eucrm.byd.com/oss/crm/c5919112-db64-48...,DED003,9.959457,48.401485,89077,+49 8331 9550 669,ulm@reisacher.de,salesAndAfterSales,"Blaubeurer Str. 110, 89077, Ulm, Baden-Württem...",None,None,None,No,,no,open,no,"[{'week': 'monday', 'hours': '8:00 am-6:00 pm'..."
8,DE,4,DES0005,BYD Schwerin Store,operation,open,https://sternauto-gruppe.de/standort/schwerin-...,https://eucrm.byd.com/oss/crm/75aae276-50f8-48...,DED002,11.347202,53.647550,19057,+49 385 48880,schwerin@sternauto.de,salesAndAfterSales,"Bremsweg 4, 19057, Schwerin, Mecklenburg-Vorpo...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '8:00 am-6:00 pm'..."
9,DE,4,DES0013,BYD Potsdam Store,operation,open,https://www.sternauto.de/standorte/nuthetal/,None,DED002,13.099167,52.357499,14558,+49 331 888000,info@autohaus-sternagel.de,salesAndAfterSales,"Arthur-Scheunert-Allee 3, 14558, Nuthetal, Bra...",None,None,None,Yes,,no,open,no,"[{'week': 'monday', 'hours': '7:00 am-6:00 pm'..."


In [99]:
df[(df['lng'] > 40) & (df['lat'] < 30)]

,country,organizationName,lng,lat,zipCode,phoneNumber,feature,address,timestamp


In [38]:
df = df[['country','organizationName','lng', 'lat','zipCode','phoneNumber','feature', 'address',]]

In [46]:
df = df.copy()
df['timestamp'] = pd.to_datetime('now')

In [9]:
import os
from time import time
from time import sleep
from datetime import date

In [15]:
def dealer_point_runner(country_list):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    for country in country_list:
        url = f'https://eu-site-api.byd.com/byd-api/eu/dealer/getFindDealer?country={country}&type=sales'
        try:
            page = requests.get(url, headers = headers)
            soup1 = BeautifulSoup(page.text, 'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

            json_text = soup2.get_text()
            data = json.loads(json_text)
            df = pd.json_normalize(data)

            df = df[['country','organizationName','lng', 'lat','zipCode','phoneNumber','feature', 'address',]]
            df = df.copy()
            df['timestamp'] = date.today()

            # correct the misordered input location data
            condition = (df['lng'] > 40) & (df['lat'] < 30)
            df = df.copy()
            df.loc[condition, ['lng', 'lat']] = df.loc[condition, ['lat', 'lng']].values

            if not os.path.isfile(r'D:\DS_road\data\web_scraping\BYD_eunew.csv'):
                df.to_csv(r'D:\DS_road\data\web_scraping\BYD_eunew.csv',header = 'column_names')
            else:
                df.to_csv(r'D:\DS_road\data\web_scraping\BYD_eunew.csv',mode = 'a',header = False)
            print(f"Successfully scraped {country}")

        except Exception as e:
            print(f"Failed for {country}: {e}")

In [17]:
country_codes = ['BE','BG','CZ','DK','DE','EE','IE','EL','ES','FR','HR','IT','CY','LV','LT','LU','HU','MT','NL','AT','PL','PT','RO','SI','SK','FI','SE','NO','CH']

for i in range(333):
    
    dealer_point_runner(country_codes)
    print("one rund completed")
    sleep(60)  #432000 = 5days
exit()

Successfully scraped BE
Failed for BG: "None of [Index(['country', 'organizationName', 'lng', 'lat', 'zipCode', 'phoneNumber',\n       'feature', 'address'],\n      dtype='object')] are in the [columns]"
Successfully scraped CZ
Successfully scraped DK
Successfully scraped DE
Successfully scraped EE
Successfully scraped IE
Failed for EL: "None of [Index(['country', 'organizationName', 'lng', 'lat', 'zipCode', 'phoneNumber',\n       'feature', 'address'],\n      dtype='object')] are in the [columns]"
Successfully scraped ES
Successfully scraped FR
Successfully scraped HR
Successfully scraped IT
Failed for CY: "None of [Index(['country', 'organizationName', 'lng', 'lat', 'zipCode', 'phoneNumber',\n       'feature', 'address'],\n      dtype='object')] are in the [columns]"
Successfully scraped LV
Successfully scraped LT
Successfully scraped LU
Successfully scraped HU
Failed for MT: "None of [Index(['country', 'organizationName', 'lng', 'lat', 'zipCode', 'phoneNumber',\n       'feature', 'a

KeyboardInterrupt: 